# Tensorflow model

In [5]:
from six.moves import cPickle as pickle
pickle_file = './' +  '1day.pickle'


with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    data = save['volume']
    del save

In [6]:
import numpy as np
from __future__ import print_function

In [8]:
fshp = data.shape
fshp

(40, 61, 61, 13)

In [10]:
data = data.reshape(fshp[0]*fshp[1]*fshp[2], fshp[3])

In [ ]:
'''
f0 = feature[:,:,:,0]
f0[f0 <= -2]=np.nan
f2 = features[:,:,:,2]
f2[f0 <= -999]=np.nan
f3 = features[:,:,:,3]
f3[f3 <= -999]=np.nan
f4 = features[:,:,:,4]
f4[f4 <= -999]=np.nan
'''

In [12]:
b = np.isnan(data).any(axis=1)
data = data[~b]
data.shape

(148840, 13)

In [14]:
labels = data[:,2:5]

In [15]:
labels.shape

(148840, 3)

In [16]:
features = np.hstack([data[:,:2], data[:,5:]])
features.shape

(148840, 10)

## dataset partition

In [20]:
class Proprocess:
    def __init__(self):
        self.dataset_standerlize = self.Dataset_standerlize()
        
    class Dataset_standerlize:
        def __init__(self):
            self.mean = 0
            self.std = 1
            
        def fit(self, dataset):
            '''The dataset should has format shape(samples_numbers)'''
            self.mean = dataset.mean(axis = 0)
            self.std = dataset.std(axis = 0)
            
        def transform(self, dataset):
            return (dataset - self.mean)/self.std
        
        def show_parameters(self):
            print (self.mean, self.std)
            
        
    def dataset_split(self, features, labels, ratio = [0.7, 0.1, 0.2], has_validate = False):
        '''The features should has format shape(samples_numbers, features), 
        the labels should has formates shape(sample_nmbers, labels)'''
        length = features.shape[0]
        length_list = range(0, length)
        if has_validate == True:
            train_len = int(length * ratio[0])
            validate_len = int(length * ratio[1]) 
            test_len = length - train_len - validate_len
            train_list = length_list[0:train_len]
            validate_list = length_list[train_len:train_len + validate_len]
            test_list = length_list[train_len + validate_len:]
            train_dataset = {
                'data' : features[train_list],
                'label' : labels[train_list],
            }
            validate_dataset= {
                'data' : features[validate_list],
                'label' : labels[validate_list],
            }
            test_dataset = {
                'data' : features[test_list],
                'label' : labels[test_list],
            }
            return train_dataset, validate_dataset, test_dataset
        if has_validate == False:
            train_len = int(length * (ratio[0]+ ratio[1]))
            test_len = length - train_len
            train_list = length_list[0:train_len]
            test_list = length_list[train_len:]
            train_dataset = {
                'data' : features[train_list],
                'label' : labels[train_list],
            }
            test_dataset = {
                'data' : features[test_list],
                'label' : labels[test_list],
            }
            return train_dataset, test_dataset

In [21]:
prep = Proprocess()

In [ ]:
train, test = prep.dataset_split(features, labels)

In [22]:
prep.dataset_standerlize.show_parameters()
prep.dataset_standerlize.fit(features)
prep.dataset_standerlize.show_parameters()

0 1
[  0.52406907   0.94947016 -87.67008972 -87.99700165 -88.31690979
 -87.22756958 -86.86695862 -94.09000397 -92.716362    -0.98706895] [  3.00981379   3.81172252  34.91673279  34.385952    33.61766815
  35.82443237  36.4488678   24.52680016  27.6547718    0.2085733 ]


## dataset normalize

In [ ]:

train_data_n = prep.dataset_standerlize.transform(train['data'])
test_data_n = prep.dataset_standerlize.transform(test['data'])

In [ ]:
train_label = train['label']
test_label = test['label']

In [23]:
features_n = prep.dataset_standerlize.transform(features)
features_n.shape

(148840, 10)

In [27]:
features_n = features_n.reshape(fshp[0], fshp[1], fshp[2], 10)
features_n.shape

(40, 61, 61, 10)

In [35]:
labels = labels.reshape(fshp[0], fshp[1], fshp[2], 3)
labels_15min_mid = labels[:, 30, 30, 2:]
labels_15min_mid.shape

(40, 1)

In [34]:
features_n_VII = features_n[:, :, :, -1:]
features_n_VII.shape

(40, 61, 61, 1)

In [56]:
def create_batch(features, offset = 0, batch_size = 4, time_duration = 8):
    if offset < 0 and offset > features.shape[0] - time_duration - batch_size:
        raise ValueError('Incorrect offset value, the offset should < batch - 1 and offset > features.shape[0]')
    shp = features.shape
    batch = [features[offset + i:offset + i + time_duration, :, :, :].reshape(1, time_duration, shp[1], shp[2], shp[3])
    for i in range(0, batch_size)]
    return np.vstack(batch)
    

In [62]:
b = create_batch(features_n_VII, batch_size = 30)
b.shape

(30, 8, 61, 61, 1)

In [70]:
c = labels_15min_mid[:30]
c.shape

(30, 1)

SyntaxError: keyword can't be an expression (<ipython-input-45-ea8e339fb74b>, line 2)

## TensorFlow model

In [40]:
import tensorflow as tf

In [ ]:
class tfmodel:
    def __init__(self):
        self.batch_size = 128
        self.steps = 25000
        
    def fit(self, train_data, train_label):
        #simple regression model train_data has shape(samples, features) and label has shape(samples, 1)
        dshp = train_data.shape
        self.create_model(dshp[1], 1)
        self.train(train_data, train_label)
        
    def predict(self, data):
        # TODO: return predict value
        with tf.Session(graph = self.graph) as session:
            tf.initialize_all_variables().run()
            print('Initialized')
            feed_dict = {self.tf_train_dataset : data}
            return self.predicted_label.eval(feed_dict = feed_dict)
        # return self.model(data)
    
    def model_init(self, feature_num, label_num):
        self.weights = tf.Variable(tf.truncated_normal([feature_num, label_num]))
        self.biases = tf.Variable(tf.zeros([label_num]))
        
    def model(self, X):
        return tf.matmul(X, self.weights) + self.biases
    
    def create_model(self, feature_num, label_num):
        self.graph = tf.Graph()
        with self.graph.as_default():
            self.tf_train_dataset = tf.placeholder(tf.float32, shape=(None, feature_num))
            self.tf_train_label = tf.placeholder(tf.float32, shape=(None, label_num))
            self.model_init(feature_num, label_num)

            

            self.predicted_label = self.model(self.tf_train_dataset)
            self.loss = tf.reduce_mean(tf.square(self.predicted_label - self.tf_train_label))

            # Learning rate decay
            global_step = tf.Variable(0)
            starter_learning_rate = 0.01
            self.learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90, staircase=True)
            self.op = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss, global_step = global_step)
    
    def train(self, train_data, train_label):
        batch_size = self.batch_size
        with tf.Session(graph = self.graph) as session:
            tf.initialize_all_variables().run()
            print('Initialized')
            for step in range(self.steps):
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_label.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_data[offset:(offset + batch_size), :]
                #print(batch_data.shape)
                batch_labels = train_label[offset:(offset + batch_size), 0].reshape(batch_size, 1)
                feed_dict = {self.tf_train_dataset : batch_data, self.tf_train_label : batch_labels}
                #session.run(predicted_label, feed_dict=feed_dict)
                l, _, r = session.run([self.loss, self.op, self.learning_rate], feed_dict=feed_dict)

                if (step % 500 == 0):
                    print('step = %d, learning rate = %f, loss = %f' % (step, r, l))

In [ ]:
model = tfmodel()

In [ ]:
model.fit(train_data_n, train_label)

In [ ]:
model.predict(test_data_n[:2000, :]).shape

# TFclass optimalize

In [86]:
class Model_base:
    def __init__(self, feature_num, label_num): 
        self.info()
        self.model_init_parameters(feature_num, label_num)
    def info(self):
        print('Using basic model:')
    
    def model(self, X):
        return tf.matmul(X, self.weights) + self.biases
    
    def model_init_parameters(self, feature_num, label_num):
        self.weights = tf.Variable(tf.truncated_normal([feature_num, label_num]))
        self.biases = tf.Variable(tf.zeros([label_num]))
        
class Model_complex(Model_base):
    
    def info(self):
        print('Using complex model:')
        
    def model(self, X):
        hidden = tf.matmul(X, self.weights['layer1']) + self.biases['layer1']
        return tf.matmul(tf.nn.relu(hidden), self.weights['layer2']) + self.biases['layer2']
    
    def model_init_parameters(self, feature_num, label_num):
        self.weights = {'layer1': tf.Variable(tf.truncated_normal([feature_num, 5])),
                       'layer2': tf.Variable(tf.truncated_normal([5, label_num])),}
        self.biases = {'layer1': tf.Variable(tf.zeros([5])),
                       'layer2': tf.Variable(tf.zeros([label_num])),}

class simple_DL():
    def __init__(self, fshp, lshp):
        self.info()
        self.model_init_parameters(fshp, lshp)
        
    def model(self, X):
        shape = tf.pack([tf.shape(X)[0], tf.shape(X)[1]*tf.shape(X)[2]*tf.shape(X)[3]])
        reshape = tf.reshape(X, shape)
        hidden = tf.matmul(reshape, self.weights['layer1']) + self.biases['layer1']
        return tf.matmul(tf.nn.relu6(hidden), self.weights['layer2']) + self.biases['layer2']
        
    def model_init_parameters(self, fshp, lshp):
        if len(fshp) != 5 and len(lshp) != 2:
            raise ValueError('Incorrect shape for fshp %s and lshp %s' % (fshp, lshp))
        feature_num = fshp[1] * fshp[2] * fshp[3] * fshp[4]
        label_num = lshp[1]
        self.weights = {'layer1': tf.Variable(tf.truncated_normal([feature_num, 61])),
                       'layer2': tf.Variable(tf.truncated_normal([61, label_num])),}
        self.biases = {'layer1': tf.Variable(tf.zeros([61])),
                       'layer2': tf.Variable(tf.zeros([label_num])),}
    
    def info(self):
        print('Using simple Deep learning model:')

In [106]:
class CNN_3D():
    def __init__(self, fshp, lshp):
        self.info()
        self.model_init_parameters(fshp, lshp)
        
    def model(self, X):
        conv = tf.nn.conv3d(X, self.weights['layer1'], [1, 1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + self.biases['layer1'])
        conv = tf.nn.conv3d(hidden, self.weights['layer2'], [1, 1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + self.biases['layer2'])
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3] * shape[4]])
        hidden = tf.nn.relu(tf.matmul(reshape, self.weights['layer3']) + self.biases['layer3'])
        return tf.matmul(hidden, self.weights['layer4']) + self.biases['layer4']
        
    def model_init_parameters(self, fshp, lshp):
        if len(fshp) != 5 and len(lshp) != 2:
            raise ValueError('Incorrect shape for fshp %s and lshp %s' % (fshp, lshp))
        patch_size = 3
        channels = fshp[4]
        depth1 = 8
        depth2 = 16
        label_num = lshp[1]
        num_hidden = 61
        self.weights = {'layer1': tf.Variable(tf.truncated_normal([patch_size, patch_size, patch_size, channels, depth1])),
                       'layer2':tf.Variable(tf.truncated_normal([patch_size, patch_size, patch_size, depth1, depth2])),
                       'layer3': tf.Variable(tf.truncated_normal([fshp[1]*fshp[2]*fshp[3]*depth2 , num_hidden], stddev=0.1)),
                       'layer4': tf.Variable(tf.truncated_normal([num_hidden, label_num], stddev=0.1)),}
                      
        self.biases = {'layer1': tf.Variable(tf.zeros([depth1])),
                       'layer2': tf.Variable(tf.zeros([depth2])),
                       'layer3': tf.Variable(tf.zeros([num_hidden])),
                       'layer4': tf.Variable(tf.zeros([label_num])),}
    
    def info(self):
        print('Using 3D CNN learning model:')

In [107]:
class tfmodel:
    def __init__(self, batch_size = 128, steps = 25000, model_class = CNN_3D):
        self.batch_size = batch_size
        self.steps = steps
        self.model_class = model_class
        
    def fit(self, train_data, train_label):
        #simple regression model train_data has shape(samples, features) and label has shape(samples, 1)
        dshp = list(train_data.shape)
        dshp[0] = None
        dshp = tuple(dshp)
        lshp = list(train_label.shape)
        lshp[0] = None
        lshp = tuple(lshp)
        self.create_model(dshp, lshp)
        self.train(train_data, train_label)
        
    def predict(self, data):
        # TODO: return predict value
        with tf.Session(graph = self.graph) as session:
            tf.initialize_all_variables().run()
            print('Initialized')
            feed_dict = {self.tf_train_dataset : data}
            return self.predicted_label.eval(feed_dict = feed_dict)
        # return self.model(data)
    
    
    def create_model(self, feature_shape, label_shape):
        
        self.graph = tf.Graph()
        with self.graph.as_default():
            self.tf_train_dataset = tf.placeholder(tf.float32, shape=feature_shape)
            self.tf_train_label = tf.placeholder(tf.float32, shape=label_shape)
            self.base = self.model_class(feature_shape, feature_shape)
            self.predicted_label = self.base.model(self.tf_train_dataset)
            self.loss = tf.reduce_mean(tf.square(self.predicted_label - self.tf_train_label))

            # Learning rate decay
            global_step = tf.Variable(0)
            starter_learning_rate = 0.01
            self.learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90, staircase=True)
            self.op = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss, global_step = global_step)
    
    def train(self, train_data, train_label):
        batch_size = self.batch_size
        with tf.Session(graph = self.graph) as session:
            tf.initialize_all_variables().run()
            print('Initialized')
            for step in range(self.steps):
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_label.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_data[offset:(offset + batch_size), :]
                #print(batch_data.shape)
                batch_labels = train_label[offset:(offset + batch_size), :]
                feed_dict = {self.tf_train_dataset : batch_data, self.tf_train_label : batch_labels}
                #session.run(predicted_label, feed_dict=feed_dict)
                l, _, r = session.run([self.loss, self.op, self.learning_rate], feed_dict=feed_dict)

                if (step % (self.steps / 50) == 0):
                    print('step = %d, learning rate = %f, loss = %f' % (step, r, l))

In [108]:
model = tfmodel(batch_size = 4, steps = 2500)

In [109]:
model.fit(b, c)

Using 3D CNN learning model:


TypeError: Expected int32, got None of type '_Message' instead.